# VIGOR Dataset: Visualize Corrected Labels

This Jupyter notebook can be used to visualize the corrected labels of the VIGOR dataset.

In [ ]:
import cv2
import ipywidgets
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os

### 1. Load Data

Below, the names of the images are obtained for the indicated city. This notebook should be placed in the folder of the VIGOR dataset, i.e., the directory should be ``DIR_TO_VIGOR_DATASET/VIGOR/VisualizeCorrectedLabels.ipynb``.

In [ ]:
city_idx           = 0
file_idx           = 0
cities             = ["Chicago", "NewYork", "SanFrancisco", "Seattle"]
files              = ["pano_label_balanced__corrected.txt", "same_area_balanced_train__corrected.txt", "same_area_balanced_test__corrected.txt"]

# folder names
ground_folder_name = "panorama"
aerial_folder_name = "satellite"
splits_name        = "splits__corrected"

# directories
dataset_dir        = os.getcwd()
ground_folder_dir  = os.path.join(dataset_dir, cities[city_idx], ground_folder_name)
aerial_folder_dir  = os.path.join(dataset_dir, cities[city_idx], aerial_folder_name)
combination_dir    = os.path.join(dataset_dir, splits_name, cities[city_idx], files[file_idx])

# get image names
ground_img_names   = list(sorted(os.listdir(ground_folder_dir)))
aerial_img_names   = list(sorted(os.listdir(aerial_folder_dir)))

print(f"{cities[city_idx]} & {files[file_idx]}: {len(ground_img_names)} ground images and {len(aerial_img_names)} aerial images found!")

### 2. Show Labels

Below, the images of the VIGOR dataset are displayed with an interactive widget. The number of image pairs that can be shown with the widget has been set to 100. The semi-positive aerial images are also displayed.

In [ ]:
def plot_image_pair(ground_folder_dir, aerial_folder_dir, combination_dir, ground_img_name):
    def get_aerial_and_deltas(ground_img_name, combination_dir):
        data_list = []
        with open(combination_dir, 'r') as file:
            for line in file.readlines():
                data = line.split(' ')
                if data[0]==ground_img_name:
                    for idx in range(4):
                        data_list.append((data[3*idx+1], float(data[3*idx+2]), float(data[3*idx+3])))
                    break
        return data_list
    
    # load images
    data_list   = get_aerial_and_deltas(ground_img_name, combination_dir)
    ground_img  = cv2.imread(os.path.join(ground_folder_dir, ground_img_name))[:,:,::-1]
    aerial_img1 = cv2.imread(os.path.join(aerial_folder_dir, data_list[0][0]))[:,:,::-1]
    aerial_img2 = cv2.imread(os.path.join(aerial_folder_dir, data_list[1][0]))[:,:,::-1]
    aerial_img3 = cv2.imread(os.path.join(aerial_folder_dir, data_list[2][0]))[:,:,::-1]
    aerial_img4 = cv2.imread(os.path.join(aerial_folder_dir, data_list[3][0]))[:,:,::-1]    
    W, H, A     = ground_img.shape[1], ground_img.shape[0], aerial_img1.shape[0]
    
    # get deltas
    delta1 = data_list[0][1:]
    delta2 = data_list[1][1:]
    delta3 = data_list[2][1:]
    delta4 = data_list[3][1:]
    
    # create plot
    fig  = plt.figure(figsize=[15, 11])
    grid = plt.GridSpec(2, 3, wspace=0.1, hspace=0.05)
    ax1  = plt.subplot(grid[0, :2])
    ax2  = plt.subplot(grid[0, 2])
    ax3  = plt.subplot(grid[1, 0])
    ax4  = plt.subplot(grid[1, 1])
    ax5  = plt.subplot(grid[1, 2])
    
    # plot ground view
    ax1.imshow(ground_img, extent=(0, W, H, 0), zorder=-10)
    ax1.set_title('Ground View', pad=10, fontsize=24)
    ax1.set_xlim(0, W)
    ax1.set_ylim(H, 0)
    
    # plot aerial view (positive)
    ax2.imshow(aerial_img1, extent=(0, A, A, 0), zorder=-10)
    ax2.set_title('Satellite View (Positive)', pad=16, fontsize=24)
    ax2.set_xlim(0, A)
    ax2.set_ylim(A, 0)
    
    # plot aerial view (semi-positive 1)
    ax3.imshow(aerial_img2, extent=(0, A, A, 0), zorder=-10)
    ax3.set_title('Semi-Positive 1', pad=10, fontsize=24)
    ax3.set_xlim(0, A)
    ax3.set_ylim(A, 0)
    
    # plot aerial view (semi-positive 2)
    ax4.imshow(aerial_img3, extent=(0, A, A, 0), zorder=-10)
    ax4.set_title('Semi-Positive 2', pad=10, fontsize=24)
    ax4.set_xlim(0, A)
    ax4.set_ylim(A, 0)
    
    # plot aerial view (semi-positive 3)
    ax5.imshow(aerial_img4, extent=(0, A, A, 0), zorder=-10)
    ax5.set_title('Semi-Positive 3', pad=10, fontsize=24)
    ax5.set_xlim(0, A)
    ax5.set_ylim(A, 0)
    
    # plot rays
    colors = ['springgreen', 'deepskyblue', 'orange', 'magenta']                            # North, East, South, West 
    xc1, yc1 = A/2-delta1[1], A/2+delta1[0]                                                 # See GitHub of VIGOR for this formula
    xc2, yc2 = A/2-delta2[1], A/2+delta2[0]                                                 # See GitHub of VIGOR for this formula
    xc3, yc3 = A/2-delta3[1], A/2+delta3[0]                                                 # See GitHub of VIGOR for this formula
    xc4, yc4 = A/2-delta4[1], A/2+delta4[0]                                                 # See GitHub of VIGOR for this formula
    
    ax1.vlines(x=0.00*W, ymin=0, ymax=H, color=colors[2], linewidth=3, zorder=10)           # South
    ax1.vlines(x=0.25*W, ymin=0, ymax=H, color=colors[3], linewidth=3, zorder=10)           # West
    ax1.vlines(x=0.50*W, ymin=0, ymax=H, color=colors[0], linewidth=3, zorder=10)           # North
    ax1.vlines(x=0.75*W, ymin=0, ymax=H, color=colors[1], linewidth=3, zorder=10)           # East
    ax1.vlines(x=1.00*W, ymin=0, ymax=H, color=colors[2], linewidth=3, zorder=10)           # South
    
    ax2.scatter(xc1, yc1, s=150, color="yellow", zorder=20)                                 # Center
    ax2.vlines(x=xc1, ymin=0, ymax=yc1, color=colors[0], linewidth=3, zorder=10)            # North
    ax2.hlines(y=yc1, xmin=xc1, xmax=A, color=colors[1], linewidth=3, zorder=10)            # East
    ax2.vlines(x=xc1, ymin=yc1, ymax=A, color=colors[2], linewidth=3, zorder=10)            # South
    ax2.hlines(y=yc1, xmin=0, xmax=xc1, color=colors[3], linewidth=3, zorder=10)            # West
    
    ax3.scatter(xc2, yc2, s=150, color="yellow", zorder=20)                                 # Center
    ax3.vlines(x=xc2, ymin=0, ymax=yc2, color=colors[0], linewidth=3, zorder=10)            # North
    ax3.hlines(y=yc2, xmin=xc2, xmax=A, color=colors[1], linewidth=3, zorder=10)            # East
    ax3.vlines(x=xc2, ymin=yc2, ymax=A, color=colors[2], linewidth=3, zorder=10)            # South
    ax3.hlines(y=yc2, xmin=0, xmax=xc2, color=colors[3], linewidth=3, zorder=10)            # West
    
    ax4.scatter(xc3, yc3, s=150, color="yellow", zorder=20)                                 # Center
    ax4.vlines(x=xc3, ymin=0, ymax=yc3, color=colors[0], linewidth=3, zorder=10)            # North
    ax4.hlines(y=yc3, xmin=xc3, xmax=A, color=colors[1], linewidth=3, zorder=10)            # East
    ax4.vlines(x=xc3, ymin=yc3, ymax=A, color=colors[2], linewidth=3, zorder=10)            # South
    ax4.hlines(y=yc3, xmin=0, xmax=xc3, color=colors[3], linewidth=3, zorder=10)            # West
    
    ax5.scatter(xc4, yc4, s=150, color="yellow", zorder=20)                                 # Center
    ax5.vlines(x=xc4, ymin=0, ymax=yc4, color=colors[0], linewidth=3, zorder=10)            # North
    ax5.hlines(y=yc4, xmin=xc4, xmax=A, color=colors[1], linewidth=3, zorder=10)            # East
    ax5.vlines(x=xc4, ymin=yc4, ymax=A, color=colors[2], linewidth=3, zorder=10)            # South
    ax5.hlines(y=yc4, xmin=0, xmax=xc4, color=colors[3], linewidth=3, zorder=10)            # West    
    
    # disable ticks
    ax1.axis('off')
    ax2.axis('off')
    ax3.axis('off')
    ax4.axis('off')
    ax5.axis('off')

    plt.show()

    # print image names
    print("Ground View:                   {}".format(ground_img_name))
    print("Aerial View (positive):        {}".format(data_list[0][0]))
    print("Aerial View (semi-positive 1): {}".format(data_list[1][0]))
    print("Aerial View (semi-positive 2): {}".format(data_list[2][0]))
    print("Aerial View (semi-positive 3): {}".format(data_list[3][0]))

In [ ]:
ipywidgets.interact(lambda idx: plot_image_pair(ground_folder_dir=ground_folder_dir,
                                                aerial_folder_dir=aerial_folder_dir,
                                                combination_dir=combination_dir,
                                                ground_img_name=ground_img_names[idx]),
                    idx=range(min(len(ground_img_names), 100)))